In [2]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [17]:
from openai import OpenAI

client = OpenAI()

# Solicita ao usuário que insira o código em R
r_code = input("Digite o código em R: ")

# Define o conteúdo da mensagem do usuário com base no código em R inserido
user_message_content = f"""Convert the R code in the following markdown code block into Python. Pay particular attention to style, and try to use imports correctly. Render all output code between "```python" markdown code blocks:

```R

{r_code}

```"""

# Cria a mensagem do usuário
user_message = {"role": "user", "content": user_message_content}

# Adiciona a mensagem do usuário à lista de mensagens
messages = [
    {"role": "system", "content": "You are a highly experienced developer working on LangChain, an open source framework that helps developers build applications with large language models (LLMs).\n\nLangChain contains abstractions and integrations for working with LLMs, chat models, vector databases, autonomous agents, and more.\n\nIt has two versions: R, Python and PySpark. You are well-versed in both languages, and help keep both versions in sync."},
    user_message
]

# Cria a conclusão com as mensagens atualizadas
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)


# Imprime a resposta gerada
print(completion.choices[0].message.content)


Digite o código em R: r
```python
# Python code will be here
```


In [18]:
from openai import OpenAI
client = OpenAI()

my_assistant = client.beta.assistants.create(
    instructions="You are a personal python tutor. When asked a question, you change the default valures and run the code to answer the question.",
    name="Python Tutor",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4",
)
print(my_assistant)


Assistant(id='asst_2cWofrcIiBe0hOKQGWBgoP3P', created_at=1712627834, description=None, file_ids=[], instructions='You are a personal python tutor. When asked a question, you change the default valures and run the code to answer the question.', metadata={}, model='gpt-4', name='Python Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')])


In [19]:
thread = client.beta.threads.create()

In [20]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=completion.choices[0].message.content
)

In [24]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Then, we use the `create_and_stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id='asst_2cWofrcIiBe0hOKQGWBgoP3P',
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Sure, Jane Doe. Feel free to provide the instruction or Python code that you want me to handle.

In [11]:
from openai import OpenAI, AssistantEventHandler
from typing_extensions import override

# Inicializa o cliente OpenAI
client = OpenAI()

# Solicita ao usuário que insira o código em R
r_code = input("Digite o código em R: ")

# Define o conteúdo da mensagem do usuário com base no código em R inserido
user_message_content = f"""Convert the R code in the following markdown code block into Python. Pay particular attention to style, and try to use imports correctly. Render all output code between "```python" markdown code blocks:

```R

{r_code}

```"""

# Cria a mensagem do usuário
user_message = {"role": "user", "content": user_message_content}

# Adiciona a mensagem do usuário à lista de mensagens
messages = [
    {"role": "system", "content": "You are a highly experienced developer working on LangChain, an open source framework that helps developers build applications with large language models (LLMs).\n\nLangChain contains abstractions and integrations for working with LLMs, chat models, vector databases, autonomous agents, and more.\n\nIt has two versions: R, Python and PySpark. You are well-versed in both languages, and help keep both versions in sync."},
    user_message
]

# Cria a conclusão com as mensagens atualizadas
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

# Imprime a resposta gerada pelo ChatGPT
print(completion.choices[0].message.content)

code_python = completion.choices[0].message.content

# Cria um assistente com base nas instruções fornecidas
my_assistant = client.beta.assistants.create(
    instructions="You are a personal python tutor. When asked a question, you change the default values and run the code to answer the question.",
    name="Python Tutor",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4",
)

# Inicializa o ID do assistente criado
assistant_id = my_assistant.id

# Define o ID do thread
thread = client.beta.threads.create()

# Cria uma mensagem com o conteúdo gerado pelo modelo anterior
message_content = completion.choices[0].message.content
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message_content
)

# Define um manipulador de eventos personalizado para o assistente
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# Cria o manipulador de eventos
event_handler = EventHandler()

# Inicia o stream de execução do assistente
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    instructions="Change the default values from {}, and do python tests and print the test and the result.".format(python_code),
    event_handler=event_handler,
) as stream:
    stream.until_done()


Digite o código em R: paises <- data.frame(   nome = c("Brasil", "Argentina", "Chile", "Uruguai", "Paraguai"),   populacao = c(211000000, 45100000, 19100000, 3510000, 7130000),   area_km2 = c(8515767, 2780400, 756102, 181034, 406752),   idh = c(0.765, 0.830, 0.851, 0.817, 0.695) )  # Visualiza as primeiras linhas do dataframe print("Primeiras linhas do dataframe:") print(head(paises))
```python
import pandas as pd

data = {
    'nome': ['Brasil', 'Argentina', 'Chile', 'Uruguai', 'Paraguai'],
    'populacao': [211000000, 45100000, 19100000, 3510000, 7130000],
    'area_km2': [8515767, 2780400, 756102, 181034, 406752],
    'idh': [0.765, 0.830, 0.851, 0.817, 0.695]
}

paises = pd.DataFrame(data)

print("Primeiras linhas do dataframe:")
print(paises.head())
```  

assistant > code_interpreter

import pandas as pd

data = {
    'nome': ['Brasil', 'Argentina', 'Chile', 'Uruguai', 'Paraguai'],
    'populacao': [211000000, 45100000, 19100000, 3510000, 7130000],
    'area_km2': [8515767, 27804